In [4]:
pip install textstat


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\HP\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



  Using cached textstat-0.7.12-py3-none-any.whl.metadata (15 kB)
  Using cached pyphen-0.17.2-py3-none-any.whl.metadata (3.2 kB)
Using cached textstat-0.7.12-py3-none-any.whl (176 kB)
Using cached pyphen-0.17.2-py3-none-any.whl (2.1 MB)

   ---------------------------------------- 0/2 [pyphen]
   ---------------------------------------- 0/2 [pyphen]
   ---------------------------------------- 0/2 [pyphen]
   ---------------------------------------- 0/2 [pyphen]
   -------------------- ------------------- 1/2 [textstat]
   -------------------- ------------------- 1/2 [textstat]
   -------------------- ------------------- 1/2 [textstat]
   -------------------- ------------------- 1/2 [textstat]
   -------------------- ------------------- 1/2 [textstat]
   -------------------- ------------------- 1/2 [textstat]
   -------------------- ------------------- 1/2 [textstat]
   -------------------- ------------------- 1/2 [textstat]
   -------------------- ------------------- 1/2 [textstat]
  

In [5]:
import pandas as pd
import numpy as np
import re
import nltk
import joblib
import textstat

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack

from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer

In [6]:
# Download NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("vader_lexicon")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
# Load datasets
fake = pd.read_csv("dataset/Fake.csv")
real = pd.read_csv("dataset/True.csv")

In [9]:
fake["label"] = 0
real["label"] = 1

In [10]:
data = pd.concat([fake, real]).sample(frac=1, random_state=42).reset_index(drop=True)
data = data[["text", "label"]]

In [11]:
# NLP tools
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
sentiment = SentimentIntensityAnalyzer()

In [12]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"\d", " ", text)
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)

data["text"] = data["text"].apply(clean_text)

In [13]:
def extra_features(texts):
    features = []
    for text in texts:
        features.append([
            len(text),
            text.count("!"),
            text.count("?"),
            textstat.flesch_reading_ease(text),
            sentiment.polarity_scores(text)["compound"]
        ])
    return np.array(features)

In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    data["text"], data["label"], test_size=0.2, random_state=42
)

In [15]:
# TF-IDF
tfidf = TfidfVectorizer(
    max_df=0.7,
    ngram_range=(1, 2),
    stop_words="english"
)

In [16]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [17]:
# Extra features
X_train_extra = extra_features(X_train)
X_test_extra = extra_features(X_test)

In [18]:
# Combine features
X_train_final = hstack([X_train_tfidf, X_train_extra])
X_test_final = hstack([X_test_tfidf, X_test_extra])

In [19]:
# Model
model = LogisticRegression(max_iter=1000, class_weight="balanced")
model.fit(X_train_final, y_train)

C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', max_iter=1000)

In [20]:
# Evaluation
pred = model.predict(X_test_final)
print("Accuracy:", accuracy_score(y_test, pred))

Accuracy: 0.9828507795100223


In [21]:
# ✅ SAVE MODELS (FIXED)
joblib.dump(model, "model/fake_news_model.pkl")
joblib.dump(tfidf, "model/tfidf_vectorizer.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'model/fake_news_model.pkl'

In [ ]:
print("✅ Model saved successfully")